In [ ]:
#FER dataset

from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow import keras 
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import tensorflow

In [ ]:
img_height, img_width = (224,224)
batch_size = 32
IMG_SHAPE = (img_height,img_width,3)
NUM_FOLDS = 5
NUM_CLASSES = 100
train_data_dir = r"/kaggle/input/fer2013/train"
valid_data_dir = r"/kaggle/input/fer2013/test"
test_data_dir = r"/kaggle/input/fer2013/test"
#test_data_dir = r"/content/drive/MyDrive/outputfolder/test"
#image.open()

In [ ]:

train_datagen = ImageDataGenerator(
                                     shear_range=0.2,zoom_range=0.2,
                                 horizontal_flip=True,
                                   validation_split=0.4,
                                    rescale=1/255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size = batch_size, 
                                                      class_mode='categorical',
                                                   shuffle = True)
                       
                                       #   subset='training')# set as training data

valid_generator = train_datagen.flow_from_directory(valid_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode='categorical')
                                                    #subset='validation') #set as validation data  
                                                                                                       

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
                                                    image_size=(img_height, img_width),
                                                    batch_size = batch_size, 
                                                     # class_mode='categorical',
                                                   shuffle = True)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(7):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
train_generator.class_indices
x=1

In [ ]:
y_train = train_generator.classes
len(y_train)

In [ ]:
nb_validation_samples = 7178

In [ ]:
dir(train_generator)
#type(train_generator.labels)
print(train_generator.classes)
#y_train = train_generator.classes
y_train = train_generator.classes
y_train.shape

In [ ]:
test_generator = train_datagen.flow_from_directory(test_data_dir,
                                                    target_size=(img_height, img_width),
                                   
                                                   batch_size = 1, 
                                                    class_mode='categorical')
                                                    #subset='validation') #set as testing data

In [ ]:
x,y = test_generator.next()
x.shape
#print(type(x))
#x

In [ ]:
!pip install matplotlib pydot graphviz
from tensorflow.keras.utils import plot_model


In [ ]:
import math
import numpy as np
import scipy
import h5py
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization  
from tensorflow.keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from matplotlib.pyplot import imshow
layer = tf.keras.layers.Dropout(0.30)
def CNN_model_1b(input_shape, outputClasses):
  
    #X_input = keras.Input(shape=(224, 224, 3))
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
   # X_2 = X
    
    
    
    X_2 = X
    #stage1:
    X = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    #stage2:
    X = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    #shortpath:
    X_2 = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X_2)
    X_2 = MaxPooling2D((3, 3), strides=(2, 2))(X_2)
    X_2 = BatchNormalization(axis=3)(X_2)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Add()([X,X_2])
    X = Activation('relu')(X)
    
    #stage3:
    X = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = layer(X)
    X = BatchNormalization(axis=3)(X)
    
    #shortpath:
    X_2 = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X_2)
    X_2 = MaxPooling2D((3, 3), strides=(2, 2))(X_2)
    X_2 = BatchNormalization(axis=3)(X_2)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Add()([X,X_2])
    X = Activation('relu')(X)
    
    X = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)    
    X = Conv2D(512, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    X = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same', 
             kernel_initializer=glorot_uniform(seed=0))(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    #avg_pooling
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    #flattening
    X = Flatten()(X)
    #fully_connected
    X = Dense(1024, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
   # X = Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Dense(train_generator.num_classes, activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs=X, name='CNN_model_1b')
    return model

In [ ]:
model = CNN_model_1b(input_shape = (224,224,3), outputClasses =7)
# Visualize the model
plot_model(model, to_file='model_plotb.png', show_shapes=True, show_layer_names=True, dpi = 1000)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','mse'])
model.summary()

In [ ]:
# 20 epochs for Custom model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

#checkpointing
#checkpoint_path = "/content/drive/MyDrive/cp-0021.ckpt.data-00000-of-00001"
#checkpoint_path = "C:/Users/DHANVANTH/SAMSUNG/Resnet/cp-0021.ckpt.data-00000-of-00001"
checkpoint_path = "G:/GAIP/Gaip_model/cp-0021.ckpt.data-00000-of-00001"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(monitor = 'val_accuracy',
   filepath=checkpoint_path, 
    verbose=1,
    save_weights_only=True,
    #save_freq=5*batch_size,
     save_best_only = True,                                                    
      mode='max')


#Reduce learning rate when a metric has stopped improving.
# Models often benefit 
#from reducing the learning rate by a factor of 2-10
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.2, patience=2, min_lr=0.00001)



# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model_history = model.fit(train_generator, epochs = 20,callbacks=[cp_callback, reduce_lr],
                          validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)
           # Pass callback to training

In [ ]:
def plot_accuracy_loss(history):
    
    fig = plt.figure(figsize=(10,5))

    plt.subplot(221)
    #plt.plot(history.history['accuracy'],'bo--', label = "train_acc")
    plt.plot(history.history['accuracy'],marker = ',',ms = 10, label = "train_acc")
    plt.plot(history.history['val_accuracy'], marker = ',', label = "val_acc")
    plt.title("(train_acc & val_acc) vs epoch")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    plt.subplot(222)
    plt.plot(history.history['loss'],marker= ',', label = "train_loss")
    plt.plot(history.history['val_loss'], marker=',' , label = "val_loss")
    plt.title("(train_loss & val_loss) vs epoch")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    f = plt.figure()
    plt.show()
  


plot_accuracy_loss(model_history)
f = plt.figure()
#f.savefig('C:/Users/DHANVANTH/SAMSUNG/resnet_50_epoch_vs_acc', bbox_inches='tight', dpi=600)

In [ ]:
#to plot confusion matrix for 100 epochs
import pandas as pd
import seaborn as sn
import tensorflow as tf


#model = model_1
filenames = test_generator.filenames
nb_samples = len(test_generator)
y_prob=[]
y_act=[]
test_generator.reset()
for _ in range(nb_samples):
  X_test,Y_test = test_generator.next()
  y_prob.append(model.predict(X_test))
  y_act.append(Y_test)

predicted_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_prob]
actual_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_act]

out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns = ['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix,cmap="YlGnBu", annot = True,fmt='d')
plt.show()
print('test accuracy: {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

In [ ]:
model.save('/kaggle/working/cnn_model1.h5')


In [ ]:
from keras.models import model_from_json

# Assuming your model is named 'model'
# Save the architecture to a JSON file
model_json = model.to_json()
with open("custom_model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the weights to an HDF5 file
model.save_weights("custom_model_weights.h5")

# Now, to load the model and its weights later
# Load the architecture from the JSON file
with open('custom_model_architecture.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

# Load the weights into the model
loaded_model.load_weights('custom_model_weights.h5')


In [ ]:
model_1 = tensorflow.keras.models.load_model('/kaggle/working/cnn_model1.h5')


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(
	actual_class, 
	predicted_class,
	target_names=class_names,digits=4 )


print(report)

In [ ]:

def create_model():

  base_model_3 = ResNet50(include_top = False, weights='imagenet',input_shape = IMG_SHAPE)
 
 #the top layer is false coz we want to classify our own classes
#we use weights from imagenet(pre-trained weights) to classify our model
  x = base_model_3.output
#taking the top layer of Resnet50 and connecting with dense layers
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024, activation='relu')(x)
  x = Dense(128, activation='relu')(x)
  predictions = Dense(train_generator.num_classes, activation='softmax')(x)
  model = Model(inputs=base_model_3.input, outputs=predictions)

  for layer in base_model_3.layers:
    layer.trainable = False
    
  opt = keras.optimizers.Adam(learning_rate=0.0001)#learning rate as 0.01
  model.compile(loss='categorical_crossentropy', optimizer=opt,
               metrics = ['accuracy','mse'])

  #model.compile(optimizer='adam', loss='categorical_crossentropy', 
            #   metrics = ['accuracy','mse'])
 # model.fit(train_generator, epochs = 10)
  return model

model = create_model()
model.summary()

In [ ]:
# 20 epochs for Resnet50 model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

#checkpointing
#checkpoint_path = "/content/drive/MyDrive/cp-0021.ckpt.data-00000-of-00001"
checkpoint_path = "L:/NUS/PROJECT/Output_folder-20220623T072421Z-001/cp-0021.ckpt.data-00000-of-00001"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(monitor = 'val_accuracy',
   filepath=checkpoint_path, 
    verbose=1,
    save_weights_only=True,
    #save_freq=5*batch_size,
     save_best_only = True,                                                    
      mode='max')


#Reduce learning rate when a metric has stopped improving.
# Models often benefit 
#from reducing the learning rate by a factor of 2-10
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.2, patience=2, min_lr=0.00001)



# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model_history = model.fit(train_generator, epochs = 20,callbacks=[cp_callback, reduce_lr],
                          validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)
           # Pass callback to training

In [ ]:
model.save('/kaggle/working/resnet50b.h5')

In [ ]:
def plot_accuracy_loss(history):
    
    fig = plt.figure(figsize=(10,5))

    plt.subplot(221)
    #plt.plot(history.history['accuracy'],'bo--', label = "train_acc")
    plt.plot(history.history['accuracy'],marker = ',',ms = 10, label = "train_acc")
    plt.plot(history.history['val_accuracy'], marker = ',', label = "val_acc")
    plt.title("(train_acc & val_acc) vs epoch")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    plt.subplot(222)
    plt.plot(history.history['loss'],marker= ',', label = "train_loss")
    plt.plot(history.history['val_loss'], marker=',' , label = "val_loss")
    plt.title("(train_loss & val_loss) vs epoch")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    f = plt.figure()
    plt.show()
  


plot_accuracy_loss(model_history)
f = plt.figure()


In [ ]:
model_1 = tensorflow.keras.models.load_model('/kaggle/working/resnet50.h5')

In [ ]:
#to plot confusion matrix for 20 epochs
import pandas as pd
import seaborn as sn
import tensorflow as tf


#model_1 = model
filenames = test_generator.filenames
nb_samples = len(test_generator)
y_prob=[]
y_act=[]
test_generator.reset()
for _ in range(nb_samples):
  X_test,Y_test = test_generator.next()
  y_prob.append(model.predict(X_test))
  y_act.append(Y_test)

predicted_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_prob]
actual_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_act]

out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns = ['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix,cmap="YlGnBu", annot = True,fmt='d')
plt.show()
print('test accuracy: {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(
	actual_class, 
	predicted_class,
	target_names=class_names,digits=4 )


print(report)

In [ ]:
#vgg19
from tensorflow.keras.applications.vgg19 import VGG19

def create_model():

  base_model_3 = VGG19(include_top = False, weights='imagenet',input_shape=IMG_SHAPE)#2ND CHANGE
 
 #the top layer is false coz we want to classify our own classes
#we use weights from imagenet(pre-trained weights) to classify our model
  x = base_model_3.output
#taking the top layer of Resnet50 and connecting with dense layers
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024, activation='relu')(x)
  predictions = Dense(train_generator.num_classes, activation='softmax')(x)
  model = Model(inputs=base_model_3.input, outputs=predictions)

  for layer in base_model_3.layers:
    layer.trainable = False
    
  opt = keras.optimizers.Adam(learning_rate=0.0001)#learning rate as 0.01
  model.compile(loss='categorical_crossentropy', optimizer=opt,
               metrics = ['accuracy','mse'])

  #model.compile(optimizer='adam', loss='categorical_crossentropy', 
            #   metrics = ['accuracy','mse'])
 # model.fit(train_generator, epochs = 10)
  return model

model = create_model()
model.summary()

#plot_model(model, show_layer_names=True, show_shapes =True, to_file='model.png', dpi=350)

In [ ]:
# 20 epochs for VGG19 model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

#checkpointing
#checkpoint_path = "/content/drive/MyDrive/cp-0021.ckpt.data-00000-of-00001"
checkpoint_path = "C:/Users/user/Desktop/vcg/cp-0021.ckpt.data-00000-of-00001"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(monitor = 'val_accuracy',
   filepath=checkpoint_path, 
    verbose=1,
    save_weights_only=True,
    #save_freq=5*batch_size,
     save_best_only = True,                                                    
      mode='max')


#Reduce learning rate when a metric has stopped improving.
# Models often benefit 
#from reducing the learning rate by a factor of 2-10
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.2, patience=2, min_lr=0.00001)



# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model_history = model.fit(train_generator, epochs = 20,callbacks=[cp_callback, reduce_lr],
                          validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size)
           # Pass callback to training

In [ ]:
def plot_accuracy_loss(history):
    
    fig = plt.figure(figsize=(10,5))

    plt.subplot(221)
    #plt.plot(history.history['accuracy'],'bo--', label = "train_acc")
    plt.plot(history.history['accuracy'],marker = ',',ms = 10, label = "train_acc")
    plt.plot(history.history['val_accuracy'], marker = ',', label = "val_acc")
    plt.title("(train_acc & val_acc) vs epoch")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    plt.subplot(222)
    plt.plot(history.history['loss'],marker= ',', label = "train_loss")
    plt.plot(history.history['val_loss'], marker=',' , label = "val_loss")
    plt.title("(train_loss & val_loss) vs epoch")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    f = plt.figure()
    plt.show()
  


plot_accuracy_loss(model_history)
f = plt.figure()

In [ ]:
model.save('/kaggle/working/vgg19b.h5')

In [ ]:
#to plot confusion matrix for 20 epochs
import pandas as pd
import seaborn as sn
import tensorflow as tf

#model = tf.keras.models.load_model("Reflection_dataset\Saved_Model\Mobilenet_reflection.h5")
#model = model_1
filenames = test_generator.filenames
nb_samples = len(test_generator)
y_prob=[]
y_act=[]
test_generator.reset()
for _ in range(nb_samples):
  X_test,Y_test = test_generator.next()
  y_prob.append(model.predict(X_test))
  y_act.append(Y_test)

predicted_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_prob]
actual_class = [list(train_generator.class_indices.keys())[i.argmax()]for i in y_act]

out_df = pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns = ['predicted_class','actual_class'])
confusion_matrix = pd.crosstab(out_df['actual_class'],out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix,cmap="YlGnBu", annot = True,fmt='d')
plt.show()
print('test accuracy: {}'.format((np.diagonal(confusion_matrix).sum()/confusion_matrix.sum().sum()*100)))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(
	actual_class, 
	predicted_class,
	target_names=class_names,digits=4 )


print(report)